# Module 1: Building Your First Agent

## Creating a UniTok Social Media Post Generator

In this notebook, we'll build an agent that can generate social media posts for Unicorn Rentals and publish them to UniTok. We'll use the Strands SDK to create our agent and add a custom tool for publishing posts to the UniTok API.

## Setup and Prerequisites

First, let's install the required packages:

In [ ]:
!pip install --upgrade strands-agents requests boto3

Now let's import the necessary libraries:

In [1]:
import json
import requests
from strands import Agent, tool
from strands.models import BedrockModel

## Creating a Custom Tool for Publishing Posts

Let's create a custom tool that will allow our agent to publish posts to the UniTok API:

In [2]:
# Define the API endpoint
# In a real deployment, this would be the actual API Gateway URL from CDK output
API_ENDPOINT = "https://08pzccde2k.execute-api.us-east-1.amazonaws.com/prod/posts"  # Replace with your actual API endpoint
# Note: The backend is implemented using Python Lambda functions

@tool
def publish_post(content: str, author: str = "Unicorn Rentals", unicorn_color: str = "rainbow", image_url: str = None) -> str:
    """
    Publish a post to the UniTok social media platform.
    
    Args:
        content (str): The text content of the post.
        author (str, optional): The author of the post. Defaults to "Unicorn Rentals".
        unicorn_color (str, optional): The color of the unicorn. Choose from: pink, blue, purple, green, yellow, or rainbow. Defaults to "rainbow".
        image_url (str, optional): URL to an image to include with the post. Defaults to None.
        
    Returns:
        str: A message indicating the post was published successfully, or an error message.
    """
    try:
        # Prepare the post data
        post_data = {
            "content": content,
            "author": author,
            "unicornColor": unicorn_color,
        }
        
        if image_url:
            post_data["imageUrl"] = image_url
            
        # Send the post to the API
        response = requests.post(API_ENDPOINT, json=post_data)
        
        # Check if the request was successful
        if response.status_code == 201:
            post_id = response.json().get("postId")
            return f"Post published successfully! Post ID: {post_id}"
        else:
            return f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}"
            
    except Exception as e:
        return f"Error publishing post: {str(e)}"

# Test the tool (commented out for now)
#result = publish_post("Testing our UniTok API!", unicorn_color="pink")
# print(result)

## Creating the Agent

Now let's create our agent using the Strands SDK. We'll use Claude 3.7 Sonnet from Amazon Bedrock as our underlying model.

In [3]:
# Define the system prompt for our agent
system_prompt = """
You are a creative social media manager for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

Your task is to create engaging social media posts for UniTok, our unicorn-themed social media platform. 
You can publish posts directly to UniTok using the publish_post tool.

Important information about Unicorn Rentals:
- We offer unicorns in various colors: pink, blue, purple, green, yellow, and rainbow (our most popular)
- Our new product feature allows customers to pick their favorite color unicorn to rent
- Our target audience includes families with children, fantasy enthusiasts, and event planners
- Our brand voice is magical, playful, and family-friendly

When creating posts:
- Keep content family-friendly and positive
- Highlight the magical experience of spending time with unicorns
- Mention the new color selection feature when appropriate
- Use emojis sparingly but effectively
- Keep posts between 50-200 characters for optimal engagement
"""

# Initialize the model
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # Add any additional model configuration here
)

# Create the agent
post_generator_agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[publish_post],
)

## Testing the Agent

Let's test our agent by asking it to create and publish a post about the new color selection feature:

In [ ]:
response = post_generator_agent("Create a post announcing our new feature that lets customers pick their favorite unicorn color.")


I'd be happy to create a post announcing Unicorn Rentals' new color selection feature! Let me craft an engaging announcement for UniTok.
Tool #1: publish_post
The post has been successfully published to UniTok! I've highlighted our exciting new color selection feature, mentioning all the available colors while encouraging engagement by asking which color our audience prefers. I kept the tone playful and magical in line with our brand voice, and included appropriate hashtags to increase visibility. The rainbow unicorn color was used as it's our most popular option.

Would you like me to create any additional posts focusing on specific benefits of this new feature or perhaps showcasing individual unicorn colors?The post has been successfully published to UniTok! I've highlighted our exciting new color selection feature, mentioning all the available colors while encouraging engagement by asking which color our audience prefers. I kept the tone playful and magical in line with our brand vo

Let's try another example, asking for a post about a specific unicorn color:

In [ ]:
response = post_generator_agent("Create a post highlighting our pink unicorns and what makes them special.")
print(response.content)

## Examining the Agent's Messages

Let's look at the messages exchanged between us and the agent:

In [ ]:
for message in post_generator_agent.messages:
    print(f"Role: {message['role']}")
    print(f"Content: {message['content']}")
    print("---")

## Analyzing Tool Usage

Let's examine how the agent used our custom tool:

In [ ]:
for message in post_generator_agent.messages:
    for content_item in message.get('content', []):
        if isinstance(content_item, dict) and 'toolUse' in content_item:
            tool_use = content_item['toolUse']
            print(f"Tool: {tool_use['name']}")
            print(f"Input: {tool_use['input']}")
            print("---")

## Viewing Posts on the UniTok Website

Now that we've created and published posts using our agent, let's see them on the UniTok website!

To view your posts:
1. Find the **UniTokUrl** from your CDK deployment output. It should look something like: `https://d123abc456def.cloudfront.net`
2. Open this URL in your web browser
3. You should see the posts that our agent has created and published, displayed in reverse chronological order

Each post shows:
- The content of the post
- The author (which we set to "Unicorn Rentals")
- The unicorn color (visualized with the appropriate color)
- The timestamp when the post was created
- The number of likes (starting at 0)

This demonstrates the end-to-end flow of our agent: it generates creative content based on our prompts, publishes it to the UniTok API, and then we can see the posts on the UniTok website.

## Conclusion

In this notebook, we've successfully created a Strands SDK agent that can generate social media posts for Unicorn Rentals and publish them to UniTok. We've learned how to:

1. Create a custom tool using the `@tool` decorator
2. Initialize a Strands agent with a BedrockModel
3. Define a system prompt to guide the agent's behavior
4. Interact with the agent to generate and publish posts
5. View the published posts on the UniTok website
6. Analyze the agent's messages and tool usage

This agent can now be used by the Unicorn Rentals marketing team to quickly create and publish engaging social media content about their unicorn rental service and new color selection feature.